In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

#preprocessing
from unidecode import unidecode
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder

#train/test split
from sklearn.model_selection import train_test_split

#vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#performance metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

#keras packages
from keras import models
from keras import layers
import tensorflow as tf
from keras import callbacks
from keras.preprocessing import text
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mafalda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#Read the data
df = pd.read_json('News_Category_Dataset_v2.json', lines = True)

df = df.drop(['link', 'date'], axis = 1)

df.head()

,category,headline,authors,short_description
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ..."


In [5]:
df['text'] = df['headline'] + df['short_description']
df.head()

,category,headline,authors,short_description,text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...


In [6]:
def preprocessing(dataframe,punctuation=False,tags=False,stemming=False,lemmatizing=False):
    """
    Function that receives a Pandas DataFrame with the texts and applies
        the chosen preprocessing techiniques.
        
    :param dataframe: a Pandas DataFrame in which the first column 
        contains the estracted texts the second column contains the
        respective authors
    :param punctuation: bool determining whether or remove punctuation
        and numbers or not (default: False)
    :param tags: bool determining whether to remove tags or not
        (default: False)
    :param stemming: bool determining whether to perform stemming or not
        (default: False)
    :param lemmatizing: bool determining whether to perform lemmatizing 
        or not (default: False)
             
    :return: Returns a list of strings which correspond to each text after
        preprocessing
    """
    
    processed_corpus = []
    
    stop_words = set(stopwords.words("english"))
    
    # for each text in the Pandas DataFrame
    for i in tqdm(range(len(dataframe))):
        text = dataframe[i]
                
        # remove punctuation
        if punctuation:
            text = re.sub('[^a-zA-Z]', ' ', text)

        # remove tags
        if tags:
            text = BeautifulSoup(text).get_text()
        
        # convert to list from str
        text = text.split()

        # stemming
        if stemming:
            stemmer = SnowballStemmer('english')
            
            # don't stem stop words so that they can still be detected
            text = [stemmer.stem(word) for word in text if not word in stop_words]
            
        if lemmatizing:
            lemmatizer = WordNetLemmatizer()
            
            text = [lemmatizer.lemmatize(word) for word in text if not word in stop_words]
        
        # convert to str from list
        text = " ".join(text)

        # save the preprocessed text on a list
        processed_corpus.append(text)
    return processed_corpus

In [7]:
# preprocess the text and save it in a Pandas Series
cleaned_text = preprocessing(
    df['text'],
    #punctuation=True,
    #tags=True,
    #stemming=True,
    #lemmatizing=True
)
df['clean_text'] = pd.Series(cleaned_text, index = df.index)

In [8]:
df.head()

,category,headline,authors,short_description,text,clean_text
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,There Were 2 Mass Shootings In Texas Last Week...
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,Will Smith Joins Diplo And Nicky Jam For The 2...
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,Hugh Grant Marries For The First Time At Age 5...
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,Julianna Margulies Uses Donald Trump Poop Bags...


In [9]:
data = df['clean_text']
target = df['category']

In [10]:
vocabulary_size = 20000
tokenizer = text.Tokenizer(num_words = vocabulary_size)
tokenizer.fit_on_texts(data)

In [11]:
sequences = tokenizer.texts_to_sequences(data)
len_sequences = [len(x) for x in sequences]
max_len = max(len_sequences)

In [12]:
max_len

241

In [13]:
data = sequence.pad_sequences(sequences, maxlen=max_len)

In [14]:
X_train,X,y_train,y = train_test_split(data,target,test_size=0.4,shuffle=True,stratify=target,random_state=0)

In [15]:
X_val,X_test,y_val,y_test = train_test_split(X,y,test_size=0.5,shuffle=True,stratify=y,random_state=0)

In [16]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],1))
X_val = X_val.reshape((X_val.shape[0],X_val.shape[1],1))

In [17]:
X_train.shape

(120511, 241, 1)

In [18]:
y_train = y_train.values.reshape((120511,1))
y_val = y_val.values.reshape((40171,1))

In [19]:
enc = OneHotEncoder()

In [20]:
y_train = enc.fit_transform(y_train)
y_val = enc.transform(y_val)

In [21]:
def nn(nodes=64,window=5,activation='relu',pooling=3,optimizer='rmsprop',loss='categorical_crossentropy'):
    np.random.seed(1)
    tf.random.set_seed(2)
    model = models.Sequential()
    model.add(layers.Conv1D(nodes,window,activation=activation,input_shape=(None,X_train.shape[-1])))
    model.add(layers.MaxPooling1D(pooling))
    #model.add(layers.Conv1D(nodes,window,activation=activation))
    model.add(layers.GRU(nodes))#,
                         #return_sequences=True))
                         #dropout=0.2,
                         #recurrent_dropout=0.2))
    #model.add(layers.GRU(nodes))
    model.add(layers.Dense(41, activation='softmax'))
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    return model

In [22]:
model = nn()

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 64)          384       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 64)                24768     
_________________________________________________________________
dense_1 (Dense)              (None, 41)                2665      
Total params: 27,817
Trainable params: 27,817
Non-trainable params: 0
_________________________________________________________________


In [24]:
callbacks_list = [
    callbacks.EarlyStopping(
        monitor = 'val_accuracy',
        patience = 10
    ),
    callbacks.ModelCheckpoint(
        filepath = 'nn.h5',
        monitor = 'val_accuracy',
        save_best_only = True
    )
]

In [25]:
history = model.fit(X_train,y_train,epochs=100,batch_size=512,callbacks=callbacks_list,validation_data=(X_val,y_val))

Train on 120511 samples, validate on 40171 samples
Epoch 1/100
120511/120511 [==============================] - 124s 1ms/step - loss: 3.3203 - accuracy: 0.1595 - val_loss: 3.2652 - val_accuracy: 0.1646
Epoch 2/100
120511/120511 [==============================] - 133s 1ms/step - loss: 3.2552 - accuracy: 0.1694 - val_loss: 3.2561 - val_accuracy: 0.1695
Epoch 3/100
120511/120511 [==============================] - 133s 1ms/step - loss: 3.2395 - accuracy: 0.1734 - val_loss: 3.2365 - val_accuracy: 0.1753
Epoch 4/100
120511/120511 [==============================] - 137s 1ms/step - loss: 3.2252 - accuracy: 0.1757 - val_loss: 3.2170 - val_accuracy: 0.1780
Epoch 5/100
120511/120511 [==============================] - 141s 1ms/step - loss: 3.2120 - accuracy: 0.1769 - val_loss: 3.2191 - val_accuracy: 0.1692
Epoch 6/100
120511/120511 [==============================] - 146s 1ms/step - loss: 3.2151 - accuracy: 0.1749 - val_loss: 3.2140 - val_accuracy: 0.1751
Epoch 7/100
120511/120511 [================